In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio

In [9]:
raster_output_25 = "data/id_carr_1km_25m.tif"
RASTER_DEM_PATH = "BDALTI/bdalti25m.tif"
RASTER_SLOPE_PATH = "BDALTI/bdalti25m_slope_deg.tif"
TOTO = "BDALTI/toto_slope_deg.tif"

In [12]:
with rasterio.open(raster_output_25) as src_tile, \
         rasterio.open(RASTER_DEM_PATH) as src_dem, \
         rasterio.open(RASTER_SLOPE_PATH) as src_slope :
    print(raster_output_25, "width : ",src_tile.width)
    print(RASTER_DEM_PATH, "width : ",src_dem.width)
    print(RASTER_SLOPE_PATH, "width : ",src_slope.width)

data/id_carr_1km_25m.tif width :  126813
BDALTI/bdalti25m.tif width :  47000
BDALTI/bdalti25m_slope_deg.tif width :  47000


In [5]:
dad.columns =['pop21','code_a','code']
dt.columns = ['pop21','code','code_a']

dad_edges=dad.groupby(['code','code_a']).pop21.sum()
dad_edges = dad_edges.reset_index()

dt_edges=dt.groupby(['code','code_a']).pop21.sum()
dt_edges = dt_edges.reset_index()

dad_edges_final = dad_edges.loc[dad_edges['pop21']>50].copy()
dt_edges_final = dt_edges.loc[dt_edges['pop21']>50].copy()

pop = pop[['COM','PMUN']]
pop.columns = ['code','pop']
dad_edges_final = dad_edges_final.merge(pop, how='left', on='code')
dad_edges_final['migra']=dad_edges_final.apply(lambda x: float(x['pop21'])/x['pop'], axis=1)
dad_edges_final = dad_edges_final[['code','code_a','migra']]

acti = acti[['CODGEO', 'P21_POP1564']]
acti.columns = ['code','pop']
dt_edges_final = dt_edges_final.merge(acti, how='left', on='code')
dt_edges_final['d_t']=dt_edges_final.apply(lambda x: float(x['pop21'])/x['pop'], axis=1)
dt_edges_final = dt_edges_final[['code','code_a','d_t']]

dt_edges_final = dt_edges_final.loc[~dt_edges_final['code_a'].isin(['YYYYY', '99999'])].copy()
dad_edges_final = dad_edges_final.loc[~dad_edges_final['code_a'].isin(['YYYYY', '99999'])].copy()

In [6]:
edges_final = dt_edges_final.merge(dad_edges_final, how='outer', on=['code','code_a']).fillna(0)

In [7]:
edges_final.to_parquet('./data_GNN/edges.parquet')

In [ ]:
routes = gpd.read_file("./data/bdtopo_routes.gpkg", columns = ['ID','IMPORTANCE','geometry'])

In [40]:
import geopandas as gpd
import pandas as pd
import torch
import numpy as np

def build_contact_opportunity_edges(gdf_communes):
    """
    Construit le graphe d'adjacence physique pondéré par la longueur de la frontière commune.
    
    Args:
        gdf_communes (GeoDataFrame): Doit contenir une colonne 'code' et 'geometry'.
                                     Les polygones doivent être valides.
    
    Returns:
        edge_index (LongTensor): [2, num_edges] Les connexions (Indices 0 à N)
        edge_attr (FloatTensor): [num_edges, 1] La longueur normalisée (Log)
        mapping_idx_code (dict): Pour retrouver le code commune à partir de l'index
    """
    print("--- Construction du Graphe d'Opportunité de Contact (Physique) ---")
    
    # 1. Préparation et Indexation
    # On travaille sur une copie pour ne pas casser l'original
    # On reset_index pour avoir des ID de nœuds propres (0, 1, 2... N) pour PyTorch
    gdf = gdf_communes.reset_index(drop=True).copy()
    
    # On crée une colonne explicite pour l'index du nœud
    gdf['node_idx'] = gdf.index
    
    # Création du mapping pour plus tard (Indispensable pour le niveau Macro)
    # Ex: {0: '01001', 1: '01002'...}
    mapping_idx_code = gdf['code'].to_dict()
    
    print("1. Nettoyage topologique léger...")
    # Buffer(0) ou très petit permet de réparer les géométries invalides 
    # et d'assurer que des polygones qui se touchent sont bien détectés
    gdf['geometry'] = gdf.geometry.buffer(0.1) 

    print("2. Détection des voisins (Spatial Join Vectorisé)...")
    # On joint le dataframe avec lui-même pour trouver les intersections
    # On garde les colonnes 'node_idx' pour savoir qui touche qui
    # suffixe _left = Source, suffixe _right = Cible
    adj = gpd.sjoin(
        gdf[['geometry', 'node_idx']], 
        gdf[['geometry', 'node_idx']], 
        how='inner', 
        predicate='intersects' # 'intersects' est plus robuste que 'touches' pour les frontières
    )
    
    # Filtre : On retire les auto-connexions (une commune se touche elle-même)
    adj = adj[adj['node_idx_left'] != adj['node_idx_right']]
    
    print(f"   -> {len(adj)} paires voisines potentielles détectées.")
    
    print("3. Calcul précis des longueurs de frontières...")
    # Optimisation : Au lieu de faire une boucle, on aligne les géométries
    # On récupère les géométries "Gauche" et "Droite" alignées sur les index du sjoin
    geom_source = gdf.loc[adj['node_idx_left'], 'geometry'].values
    geom_target = gdf.loc[adj['node_idx_right'], 'geometry'].values
    
    # Intersection vectorielle (GeoSeries vs GeoSeries)
    # Cela génère des LineString (la frontière commune)
    gs_source = gpd.GeoSeries(geom_source)
    gs_target = gpd.GeoSeries(geom_target)
    intersections = gs_source.intersection(gs_target)
    
    # Calcul de la longueur en mètres (si projection métrique type Lambert 93)
    lengths = intersections.length
    
    # 4. Nettoyage final
    # On ne garde que les frontières qui ont une longueur significative (> 1 mètre)
    # Cela élimine les points de contact uniques ou les erreurs de topologie
    mask_np = (lengths > 1.0).values 
    
    final_src = adj['node_idx_left'].values[mask_np]
    final_dst = adj['node_idx_right'].values[mask_np]
    final_attr = lengths.values[mask_np]
    
    print(f"   -> {len(final_src)} arêtes physiques validées (Longueur > 1m).")

    # 5. Conversion en Tenseurs PyTorch
    # Format edge_index : [[Sources], [Cibles]]
    edge_index = torch.tensor([final_src, final_dst], dtype=torch.long)
    
    # Format edge_attr : Normalisation Logarithmique
    # Pourquoi ? Une frontière peut faire 10m ou 50km. 
    # Le réseau apprend mieux avec des valeurs compressées (ex: entre 0 et 10) qu'avec 50000.
    # log1p calcule log(1 + x) pour gérer les petites valeurs proprement.
    edge_attr = torch.tensor(final_attr, dtype=torch.float).log1p().unsqueeze(1)
    
    return edge_index, edge_attr, mapping_idx_code


def add_road_crossings(edge_index, gdf_communes, gdf_routes):
    """
    Calcule la perméabilité routière des frontières.
    Utilise un index spatial pour la rapidité.
    """
    
    # --- 1. Préparation des données Routes ---
    print("1. Préparation des routes et de l'index spatial...")
    
    # On s'assure que l'index spatial existe (GeoPandas le crée à la volée sinon, mais c'est mieux d'être explicite)
    # L'index permet de faire des requêtes spatiales instantanées
    routes_sindex = gdf_routes.sindex 
    
    # Feature Engineering : Pondération (si colonne NATURE existe)
    # Adaptez les noms de colonnes selon votre version de la BD TOPO
    if 'NATURE' in gdf_routes.columns:
        poids_route = {
            '1': 10.0,
            '2': 8.0,
            '3': 5.0,
            '4': 3.0,
            '5': 1.0,
            '6': 0.0,
        }
        # On map et on remplit les inconnus par 1.0 (Route standard)
        weights = gdf_routes['IMPORTANCE'].map(poids_route).fillna(1.0).values
    else:
        # Si pas de colonne NATURE, on compte juste le nombre de routes (poids 1)
        weights = np.ones(len(gdf_routes))
    
    # On stocke les poids dans le dataframe pour y accéder facilement
    gdf_routes['weight_calc'] = weights

    # --- 2. Préparation des géométries Communes ---
    # Pour accéder rapidement aux polygones par leur index numpy
    geoms_communes = gdf_communes.geometry.values
    
    # On récupère les indices source/cible du graphe calculé précédemment
    src_indices = edge_index[0].numpy()
    dst_indices = edge_index[1].numpy()
    
    crossings_scores = []
    
    print(f"2. Calcul des traversées pour {len(src_indices)} frontières...")
    
    # --- 3. Boucle optimisée ---
    for i, (s, d) in enumerate(zip(src_indices, dst_indices)):
        
        # A. Récupérer les deux polygones
        poly_s = geoms_communes[s]
        poly_d = geoms_communes[d]
        
        # B. Calculer la ligne frontière (Intersection)
        # Note : intersection() peut renvoyer GeometryCollection ou MultiLineString
        boundary = poly_s.intersection(poly_d)
        
        # Cas limites : Si l'intersection est vide ou juste un point
        if boundary.is_empty or boundary.geom_type == 'Point' or boundary.length < 1.0:
            crossings_scores.append(0.0)
            continue
            
        # C. LE PRÉ-FILTRE (Spatial Index Query)
        # C'est ici que la magie opère. Au lieu de tester toutes les routes,
        # on demande à l'index : "Donne-moi les ID des routes dont la boîte (bbox) touche la frontière"
        possible_matches_index = list(routes_sindex.query(boundary, predicate='intersects'))
        
        if not possible_matches_index:
            crossings_scores.append(0.0)
            continue
            
        # D. Sélection des candidates
        # On ne charge que les quelques routes candidates
        candidate_roads = gdf_routes.iloc[possible_matches_index]
        
        # E. Intersection précise (Géométrique)
        # On vérifie si ça coupe vraiment la ligne (et pas juste passer à côté dans la bbox)
        real_intersections = candidate_roads[candidate_roads.intersects(boundary)]
        
        if real_intersections.empty:
            crossings_scores.append(0.0)
        else:
            # F. Somme des poids
            score = real_intersections['weight_calc'].sum()
            crossings_scores.append(score)
            
        # Petit log pour suivre l'avancement (tous les 1000 couples)
        if i % 1000 == 0:
            print(f"   -> Traité {i}/{len(src_indices)} frontières...")

    return crossings_scores

In [3]:
gdf_communes = gpd.read_file("data/commune_francemetro_2023.gpkg")
gdf_routes = gpd.read_file('data/bdtopo_routes.gpkg', columns = ['ID', 'IMPORTANCE', 'geometry'])

In [41]:
# ÉTAPE 1 : Le Squelette Physique
# On calcule qui se touche et la longueur de la frontière
edge_index_phys, attr_longueur, mapping = build_contact_opportunity_edges(gdf_communes)

--- Construction du Graphe d'Opportunité de Contact (Physique) ---
1. Nettoyage topologique léger...
2. Détection des voisins (Spatial Join Vectorisé)...
   -> 207748 paires voisines potentielles détectées.
3. Calcul précis des longueurs de frontières...
   -> 204284 arêtes physiques validées (Longueur > 1m).


In [42]:
# ÉTAPE 2 : La Perméabilité
# On réinjecte ce squelette (edge_index_phys) pour ne calculer les routes QUE sur les frontières existantes
# (Sinon on calculerait des millions d'intersections inutiles)
liste_poids_routes = add_road_crossings(edge_index_phys, gdf_communes, gdf_routes)



1. Préparation des routes et de l'index spatial...
2. Calcul des traversées pour 204284 frontières...
   -> Traité 0/204284 frontières...
   -> Traité 1000/204284 frontières...
   -> Traité 2000/204284 frontières...
   -> Traité 3000/204284 frontières...
   -> Traité 4000/204284 frontières...
   -> Traité 5000/204284 frontières...
   -> Traité 6000/204284 frontières...
   -> Traité 7000/204284 frontières...
   -> Traité 8000/204284 frontières...
   -> Traité 9000/204284 frontières...
   -> Traité 10000/204284 frontières...
   -> Traité 11000/204284 frontières...
   -> Traité 12000/204284 frontières...
   -> Traité 13000/204284 frontières...
   -> Traité 14000/204284 frontières...
   -> Traité 15000/204284 frontières...
   -> Traité 16000/204284 frontières...
   -> Traité 17000/204284 frontières...
   -> Traité 18000/204284 frontières...
   -> Traité 19000/204284 frontières...
   -> Traité 20000/204284 frontières...
   -> Traité 21000/204284 frontières...
   -> Traité 22000/204284 front

In [43]:
# ÉTAPE 3 : La Fusion (Crucial !)
# On transforme la liste des routes en Tenseur PyTorch
attr_routes = torch.tensor(liste_poids_routes, dtype=torch.float).unsqueeze(1)


In [44]:
# On colle les deux attributs côte à côte pour faire le vecteur final de l'arête physique
# Résultat : [Longueur, Nb_Routes] pour chaque arête
final_edge_attr = torch.cat([attr_longueur, attr_routes], dim=1)

print("Vecteur d'arête physique prêt :", final_edge_attr.shape)
# Doit afficher : [Nombre_Aretes, 2]

Vecteur d'arête physique prêt : torch.Size([204284, 2])


In [45]:
final_edge_attr

tensor([[ 8.7458, 26.0000],
        [ 9.2431, 23.0000],
        [ 8.6594, 12.0000],
        ...,
        [ 9.0923,  0.0000],
        [ 8.4743,  2.0000],
        [ 8.9491,  7.0000]])

In [ ]:
gdf = gdf_communes.reset_index(drop=True).copy()
    
# On crée une colonne explicite pour l'index du nœud
gdf['node_idx'] = gdf.index

# Création du mapping pour plus tard (Indispensable pour le niveau Macro)
# Ex: {0: '01001', 1: '01002'...}
mapping_idx_code = gdf['code'].to_dict()

print("1. Nettoyage topologique léger...")
# Buffer(0) ou très petit permet de réparer les géométries invalides 
# et d'assurer que des polygones qui se touchent sont bien détectés
gdf['geometry'] = gdf.geometry.buffer(0.1) 

print("2. Détection des voisins (Spatial Join Vectorisé)...")
# On joint le dataframe avec lui-même pour trouver les intersections
# On garde les colonnes 'node_idx' pour savoir qui touche qui
# suffixe _left = Source, suffixe _right = Cible
adj = gpd.sjoin(
    gdf[['geometry', 'node_idx']], 
    gdf[['geometry', 'node_idx']], 
    how='inner', 
    predicate='intersects' # 'intersects' est plus robuste que 'touches' pour les frontières
)

# Filtre : On retire les auto-connexions (une commune se touche elle-même)
adj = adj[adj['node_idx_left'] != adj['node_idx_right']]

print(f"   -> {len(adj)} paires voisines potentielles détectées.")

print("3. Calcul précis des longueurs de frontières...")
# Optimisation : Au lieu de faire une boucle, on aligne les géométries
# On récupère les géométries "Gauche" et "Droite" alignées sur les index du sjoin
geom_source = gdf.loc[adj['node_idx_left'], 'geometry'].values
geom_target = gdf.loc[adj['node_idx_right'], 'geometry'].values

# Intersection vectorielle (GeoSeries vs GeoSeries)
# Cela génère des LineString (la frontière commune)
gs_source = gpd.GeoSeries(geom_source)
gs_target = gpd.GeoSeries(geom_target)
intersections = gs_source.intersection(gs_target)

# Calcul de la longueur en mètres (si projection métrique type Lambert 93)
lengths = intersections.length

# 4. Nettoyage final
# On ne garde que les frontières qui ont une longueur significative (> 1 mètre)
# Cela élimine les points de contact uniques ou les erreurs de topologie
mask_np = (lengths > 1.0).values 
    
# 2. On filtre les tableaux Numpy (positionnel strict)
# On prend les .values D'ABORD, et on applique le masque ENSUITE
final_src = adj['node_idx_left'].values[mask_np]
final_dst = adj['node_idx_right'].values[mask_np]
final_attr = lengths.values[mask_np]

1. Nettoyage topologique léger...
2. Détection des voisins (Spatial Join Vectorisé)...
   -> 207748 paires voisines potentielles détectées.
3. Calcul précis des longueurs de frontières...


In [37]:
mask_np = (lengths > 1.0).values 
    
# 2. On filtre les tableaux Numpy (positionnel strict)
# On prend les .values D'ABORD, et on applique le masque ENSUITE
final_src = adj['node_idx_left'].values[mask_np]
final_dst = adj['node_idx_right'].values[mask_np]
final_attr = lengths.values[mask_np]

In [38]:
print(f"Check tailles : Src={len(final_src)}, Attr={len(final_attr)}")

Check tailles : Src=204284, Attr=204284


In [32]:
final_attr

array([1.00045300e+00, 1.00045300e+00, 1.00110600e+00, ...,
       7.61221020e+04, 8.76085924e+04, 8.76085924e+04], shape=(204284,))